In [3]:
import numpy as np
import pandas as pd
import optuna
import logging
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Modelli
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

# Stacking
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV

# Configurazione Silenziosa
optuna.logging.set_verbosity(optuna.logging.WARNING)
import warnings
warnings.filterwarnings('ignore')

# ---------------------------------------------------------
# 1. PREPARAZIONE DATI
# ---------------------------------------------------------
print("1. Caricamento e split dati...")
data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Split Principale: 80% Train (per Optuna + Stacking), 20% Test (Hold-out finale)
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split Secondario: Solo per Optuna (per non overfittare sul train set intero durante il tuning)
X_opt_train, X_opt_val, y_opt_train, y_opt_val = train_test_split(
    X_train_full, y_train_full, test_size=0.25, random_state=42
)

# ---------------------------------------------------------
# 2. OTTIMIZZAZIONE OPTUNA (LIGHT)
# ---------------------------------------------------------
print("2. Avvio Ottimizzazione Iperparametri (20 trial per modello)...")

# --- XGBoost ---
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'n_jobs': -1,
        'random_state': 42,
    }
    model = XGBRegressor(**params)
    model.fit(X_opt_train, y_opt_train)
    preds = model.predict(X_opt_val)
    return np.sqrt(mean_squared_error(y_opt_val, preds))

study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(objective_xgb, n_trials=20, show_progress_bar=True)
best_xgb_params = study_xgb.best_params
best_xgb_params.update({'n_jobs': -1, 'random_state': 42}) # Reinseriamo params statici

# --- CatBoost ---
def objective_cat(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 500, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'verbose': 0,
        'allow_writing_files': False,
        'random_state': 42
    }
    model = CatBoostRegressor(**params)
    model.fit(X_opt_train, y_opt_train)
    preds = model.predict(X_opt_val)
    return np.sqrt(mean_squared_error(y_opt_val, preds))

study_cat = optuna.create_study(direction='minimize')
study_cat.optimize(objective_cat, n_trials=20, show_progress_bar=True)
best_cat_params = study_cat.best_params
best_cat_params.update({'verbose': 0, 'allow_writing_files': False, 'random_state': 42})

# --- LightGBM ---
def objective_lgbm(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', -1, 15),
        'n_jobs': -1,
        'verbose': -1,
        'random_state': 42
    }
    model = LGBMRegressor(**params)
    model.fit(X_opt_train, y_opt_train)
    preds = model.predict(X_opt_val)
    return np.sqrt(mean_squared_error(y_opt_val, preds))

study_lgbm = optuna.create_study(direction='minimize')
study_lgbm.optimize(objective_lgbm, n_trials=20, show_progress_bar=True)
best_lgbm_params = study_lgbm.best_params
best_lgbm_params.update({'n_jobs': -1, 'verbose': -1, 'random_state': 42})

print(f"\nBest RMSE Val -> XGB: {study_xgb.best_value:.4f} | CAT: {study_cat.best_value:.4f} | LGBM: {study_lgbm.best_value:.4f}")

# ---------------------------------------------------------
# 3. COSTRUZIONE E TRAINING STACKING REGRESSOR
# ---------------------------------------------------------
print("\n3. Configurazione Stacking Regressor con i migliori parametri...")

# Definizione Level 0 (Base Models)
estimators = [
    ('xgb', XGBRegressor(**best_xgb_params)),
    ('cat', CatBoostRegressor(**best_cat_params)),
    ('lgbm', LGBMRegressor(**best_lgbm_params))
]

# Definizione Level 1 (Meta Model)
# RidgeCV trova automaticamente la regolarizzazione (alpha) migliore
meta_learner = RidgeCV(alphas=[0.1, 1.0, 10.0])

stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=meta_learner,
    cv=5,            # 5-Fold per generare le previsioni out-of-fold robuste
    n_jobs=-1,
    passthrough=False 
)

print("Addestramento Stacking (questo passaggio ri-addestra i base models su 5 fold)...")
stacking_regressor.fit(X_train_full, y_train_full)

# ---------------------------------------------------------
# 4. VALUTAZIONE FINALE
# ---------------------------------------------------------
print("\n4. Valutazione sul Test Set (Hold-out)...")

# Predizione
preds = stacking_regressor.predict(X_test)

# Metriche
rmse = np.sqrt(mean_squared_error(y_test, preds))
r2 = r2_score(y_test, preds)

print("-" * 40)
print(f"STACKING FINAL RMSE: {rmse:.4f}")
print(f"STACKING FINAL R2:   {r2:.4f}")
print("-" * 40)

# Ispezione Pesi Meta-Learner
print("Pesi assegnati dal Meta-Modello (Ridge):")
print(f"Intercept: {stacking_regressor.final_estimator_.intercept_:.4f}")
model_names = ['XGBoost', 'CatBoost', 'LightGBM']
coeffs = stacking_regressor.final_estimator_.coef_

for name, coef in zip(model_names, coeffs):
    print(f"{name:<10}: {coef:.4f}")

1. Caricamento e split dati...
2. Avvio Ottimizzazione Iperparametri (20 trial per modello)...


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]


Best RMSE Val -> XGB: 0.4549 | CAT: 0.4459 | LGBM: 0.4534

3. Configurazione Stacking Regressor con i migliori parametri...
Addestramento Stacking (questo passaggio ri-addestra i base models su 5 fold)...

4. Valutazione sul Test Set (Hold-out)...
----------------------------------------
STACKING FINAL RMSE: 0.4298
STACKING FINAL R2:   0.8590
----------------------------------------
Pesi assegnati dal Meta-Modello (Ridge):
Intercept: -0.0167
XGBoost   : 0.2949
CatBoost  : 0.5119
LightGBM  : 0.2012


In [11]:
import numpy as np
import pandas as pd
import optuna
import warnings
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Modelli
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

# Stacking
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV, ElasticNetCV, LassoCV

In [12]:

# Configurazione Silenziosa
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings('ignore')

# ---------------------------------------------------------
# 1. CARICAMENTO E PULIZIA AVANZATA (IQR)
# ---------------------------------------------------------
print("1. Data Loading & Statistical Cleaning...")

data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="MedHouseVal")
df = pd.concat([X, y], axis=1)

# Rimuoviamo il cap artificiale del target (valori >= 5.0)
df = df[df['MedHouseVal'] < 5.0]

def remove_outliers_iqr(df, columns):
    """
    Rimuove gli outlier usando la regola dell'IQR (Interquartile Range).
    Mantiene solo i dati tra Q1 - 1.5*IQR e Q3 + 1.5*IQR.
    """
    df_clean = df.copy()
    indices_to_drop = []
    
    for col in columns:
        Q1 = df_clean[col].quantile(0.30)
        Q3 = df_clean[col].quantile(0.70)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Identifichiamo gli indici da rimuovere per questa colonna
        outliers = df_clean[(df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)].index
        indices_to_drop.extend(outliers)
    
    # Rimuoviamo i duplicati degli indici e facciamo il drop
    indices_to_drop = list(set(indices_to_drop))
    df_clean = df_clean.drop(indices_to_drop)
    return df_clean

# Applichiamo IQR solo su feature "fisiche" soggette a errori di input o anomalie estreme
cols_to_clean = ['AveRooms', 'AveBedrms', 'AveOccup', 'MedInc']
original_len = len(df)
df = remove_outliers_iqr(df, cols_to_clean)
print(f"Dataset pulito con IQR: {len(df)} righe (Rimossi {original_len - len(df)} outlier)")

# ---------------------------------------------------------
# 2. FEATURE ENGINEERING AVANZATA
# ---------------------------------------------------------
print("2. Creazione Feature (Distanze, Rapporti, Cluster)...")

# A. Feature basate su Domain Knowledge
df['Bedrms_per_Room'] = df['AveBedrms'] / df['AveRooms']
df['Rooms_per_Person'] = df['AveRooms'] / df['AveOccup']
# Interazione Reddito * Stanze (Capacità di spesa per spazio)
df['Wealth_Capacity'] = df['MedInc'] * df['AveRooms']

# B. Feature Geospaziali: Distanza dai centri economici
# Coordinate approssimative (Lat, Lon)
sf_coords = (37.7749, -122.4194)
la_coords = (34.0522, -118.2437)

def dist_calc(row, city_coords):
    # Distanza Euclidea semplificata (sufficiente per ML su scala locale)
    return np.sqrt((row['Latitude'] - city_coords[0])**2 + (row['Longitude'] - city_coords[1])**2)

df['Dist_SF'] = df.apply(lambda x: dist_calc(x, sf_coords), axis=1)
df['Dist_LA'] = df.apply(lambda x: dist_calc(x, la_coords), axis=1)
# Feature: Distanza minima dal centro metropolitano più vicino
df['Dist_Min_Metro'] = df[['Dist_SF', 'Dist_LA']].min(axis=1)

def point_line_distance(x0, y0, a, b, c):
    return abs(a * x0 + b * y0 + c) / np.sqrt(a*a + b*b)
df["DistToCoast"] = point_line_distance(df['Longitude'], df['Latitude'], 1.25, 1, 116)

# Separazione X e y
X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

df['IncTot'] = df['MedInc'] * df['Population']
df['IncPerPers'] = df['MedInc'] / df['Population']
df['PopulationPerHousehold'] = df['Population'] / df['AveOccup'] 

df['MedInc_log'] = np.log1p(df['MedInc'])          # log(MedInc + 1)
df['Population_log'] = np.log1p(df['Population'])  # log(Population + 1)
df['AveRooms_log'] = np.log1p(df['AveRooms'])
df['AveBedrms_log'] = np.log1p(df['AveBedrms'])

# Split Train/Test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# C. KMeans Clustering (Gestito correttamente per evitare Leakage)
# Fit solo su Train, Transform su Train e Test
print("    -> Generazione Cluster Geografici (KMeans)...")
scaler_geo = StandardScaler()
kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)

# Prendiamo solo lat/long
train_geo = X_train_full[['Latitude', 'Longitude']]
test_geo = X_test[['Latitude', 'Longitude']]

# Scaliamo (KMeans è sensibile alla scala)
train_geo_scaled = scaler_geo.fit_transform(train_geo)
test_geo_scaled = scaler_geo.transform(test_geo)

# Creiamo la feature cluster
X_train_full['Geo_Cluster'] = kmeans.fit_predict(train_geo_scaled)
X_test['Geo_Cluster'] = kmeans.predict(test_geo_scaled)

# Assicuriamoci che Geo_Cluster sia trattata come categorica o intera
X_train_full['Geo_Cluster'] = X_train_full['Geo_Cluster'].astype(int)
X_test['Geo_Cluster'] = X_test['Geo_Cluster'].astype(int)


# Split per Optuna (dal Train set pulito)
X_opt_train, X_opt_val, y_opt_train, y_opt_val = train_test_split(
    X_train_full, y_train_full, test_size=0.25, random_state=42
)

# ---------------------------------------------------------
# 3. OTTIMIZZAZIONE OPTUNA
# ---------------------------------------------------------
print("\n3. Tuning Iperparametri...")

# --- XGBoost ---
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 600, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5, log = True),
        'max_depth': trial.suggest_int('max_depth', 4, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'n_jobs': -1,
        'random_state': 42,
    }
    model = XGBRegressor(**params)
    model.fit(X_opt_train, y_opt_train)
    return np.sqrt(mean_squared_error(y_opt_val, model.predict(X_opt_val)))

study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(objective_xgb, n_trials=15) # 15 trial per velocità
best_xgb = study_xgb.best_params
best_xgb.update({'n_jobs': -1, 'random_state': 42})

# --- CatBoost ---
def objective_cat(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 600, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5, log = True),
        'depth': trial.suggest_int('depth', 4, 7),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'verbose': 0,
        'allow_writing_files': False,
        'random_state': 42
    }
    model = CatBoostRegressor(**params)
    model.fit(X_opt_train, y_opt_train, cat_features=['Geo_Cluster'])
    return np.sqrt(mean_squared_error(y_opt_val, model.predict(X_opt_val)))

study_cat = optuna.create_study(direction='minimize')
study_cat.optimize(objective_cat, n_trials=15)
best_cat = study_cat.best_params
best_cat.update({'verbose': 0, 'allow_writing_files': False, 'random_state': 42})

# --- LightGBM ---
def objective_lgbm(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 600, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5, log = True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'n_jobs': -1,
        'verbose': -1,
        'random_state': 42
    }
    model = LGBMRegressor(**params)
    # LightGBM gestisce le categorie (Geo_Cluster) se specificate
    cat_features = ['Geo_Cluster']
    X_train_lgbm = X_opt_train.copy()
    X_val_lgbm = X_opt_val.copy()
    X_train_lgbm[cat_features] = X_train_lgbm[cat_features].astype('category')
    X_val_lgbm[cat_features] = X_val_lgbm[cat_features].astype('category')
    
    model.fit(X_train_lgbm, y_opt_train, categorical_feature=cat_features)
    return np.sqrt(mean_squared_error(y_opt_val, model.predict(X_val_lgbm)))

study_lgbm = optuna.create_study(direction='minimize')
study_lgbm.optimize(objective_lgbm, n_trials=15)
best_lgbm = study_lgbm.best_params
best_lgbm.update({'n_jobs': -1, 'verbose': -1, 'random_state': 42})

print(f"Best RMSE Val -> XGB: {study_xgb.best_value:.4f} | CAT: {study_cat.best_value:.4f} | LGBM: {study_lgbm.best_value:.4f}")

# ---------------------------------------------------------
# 4. STACKING REGRESSOR
# ---------------------------------------------------------
print("\n4. Addestramento Stacking Ensemble (CV=5)...")

# Prepariamo i dati per Stacking (gestione delle feature categoriali per lgbm e cat)
X_train_stack = X_train_full.copy()
X_test_stack = X_test.copy()
cat_features = ['Geo_Cluster']
X_train_stack[cat_features] = X_train_stack[cat_features].astype('category')
X_test_stack[cat_features] = X_test_stack[cat_features].astype('category')

estimators = [
    ('xgb', XGBRegressor(**best_xgb, enable_categorical=True)),
    ('cat', CatBoostRegressor(**best_cat, cat_features=cat_features)),
    ('lgbm', LGBMRegressor(**best_lgbm, categorical_feature=cat_features))
]





1. Data Loading & Statistical Cleaning...
Dataset pulito con IQR: 15203 righe (Rimossi 4445 outlier)
2. Creazione Feature (Distanze, Rapporti, Cluster)...
    -> Generazione Cluster Geografici (KMeans)...

3. Tuning Iperparametri...
Best RMSE Val -> XGB: 0.3595 | CAT: 0.3643 | LGBM: 0.3645

4. Addestramento Stacking Ensemble (CV=5)...


In [13]:
# RidgeCV usa la Generalized Cross-Validation per trovare l'alpha ottimale in modo efficiente

meta_learner = RidgeCV(alphas=[0.1, 0.4, 1.0, 4.0, 10.0])

stacking = StackingRegressor(
    estimators=estimators,
    final_estimator=meta_learner,
    cv=5,
    n_jobs=-1,
    passthrough=False 
)

# Addestramento finale su tutto il set di training
stacking.fit(X_train_stack, y_train_full)

# ---------------------------------------------------------
# 5. RISULTATI FINALI, COEFFICIENTI e FEATURE IMPORTANCE
# ---------------------------------------------------------
print("\n5. Valutazione Finale, Coefficienti Stacking e Analisi Importanza Feature...")

# 5.1. Calcolo e Stampa delle Metriche Finali
y_pred = stacking.predict(X_test_stack)

final_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
final_r2 = r2_score(y_test, y_pred)

print(f"\n--- RISULTATI FINALI STACKING ENSEMBLE ---")
print(f"Final RMSE (Test Set): {final_rmse:.4f}")
print(f"Final R-squared (Test Set): {final_r2:.4f}")
print("------------------------------------------")


# 5.2. Stampa dei Coefficienti del Meta-Learner (RidgeCV)
meta_coef = stacking.final_estimator_.coef_
base_names = [name for name, _ in stacking.estimators]

print("\n--- COEFFICIENTI DEL MODELLO DI STACKING (RIDGE CV) ---")
for name, coef in zip(base_names, meta_coef):
    print(f"Coefficiente {name.upper():<4}: {coef:.4f}")
print("-------------------------------------------------------")
# Nota: La somma dei coefficienti dovrebbe essere vicina a 1, indicando una media ponderata.


# 5.3. Analisi Importanza Feature
xgb_temp = stacking.estimators_[0]
feature_names = X_train_full.columns 

importances = pd.Series(xgb_temp.feature_importances_, index=feature_names)

print("\nTop 5 Feature più importanti (Basate su XGBoost Base Estimator):")
print(importances.sort_values(ascending=False).head(5))


5. Valutazione Finale, Coefficienti Stacking e Analisi Importanza Feature...

--- RISULTATI FINALI STACKING ENSEMBLE ---
Final RMSE (Test Set): 0.3525
Final R-squared (Test Set): 0.8487
------------------------------------------

--- COEFFICIENTI DEL MODELLO DI STACKING (RIDGE CV) ---
Coefficiente XGB : 0.3845
Coefficiente CAT : 0.3632
Coefficiente LGBM: 0.2627
-------------------------------------------------------

Top 5 Feature più importanti (Basate su XGBoost Base Estimator):
Geo_Cluster         0.318957
DistToCoast         0.124726
MedInc              0.083909
Rooms_per_Person    0.064340
Dist_Min_Metro      0.055833
dtype: float32
